In [1]:
import pandas as pd
import networkx as nx
import json
from IPython.core.display import display, HTML
from string import Template
import pendulum
import copy

In [2]:
tx = pd.read_csv('./nate_transactions.csv')

In [3]:
tx

,transaction_hash,to_address,from_address,block_timestamp,value,from_is_contract,to_is_contract,nft_details
0,0x009f821f75c9c75a27eddbf0a30f5039dab322e08f31...,0x881d40237659c251811cec9c364ef91dc08d300c,0x2e3922be905a3314fe5e16b44a3e878bae69c150,2021-08-16T23:36:55.000Z,0.000000e+00,NaN,True,NaN
1,0x009f821f75c9c75a27eddbf0a30f5039dab322e08f31...,0x2e3922be905a3314fe5e16b44a3e878bae69c150,0x74de5d4fcbf63e00296fd95d33236b9794016631,2021-08-16T23:36:55.000Z,1.837500e+17,True,NaN,NaN
2,0x00d2b2ba7b09445f49ca8992266acf54fd8e0fd603d4...,0x54ed0feff1467f9d13587dfd39226d08ee1a9606,0x0660459b2b658b232f3db6adff5580e7558f60e6,2021-05-26T22:14:22.000Z,0.000000e+00,NaN,True,NaN
3,0x01627ee26a1ac6ec6a25b6fafcc148e4da59349fe794...,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,0x0660459b2b658b232f3db6adff5580e7558f60e6,2021-03-13T14:27:11.000Z,1.000000e+18,NaN,True,NaN
4,0x0168a5dafde259a80cb5d77e6c9ff0d446190ddb47d2...,0x8d33303023723de93b213da4eb53be890e747c63,0xf1b30855994a95e7e2024aa44d986dc20fa6f617,2021-07-16T04:57:37.000Z,8.700000e+16,NaN,True,NaN
...,...,...,...,...,...,...,...,...
31706,0xfd46e6a48f3b6d7e0fb34f206999e51008d7b1cad632...,0xe96a1b303a1eb8d04fb973eb2b291b8d591c8f72,0xa6d954d08877f8ce1224f6bfb83484c7d3abf8e9,2018-03-23T09:26:30.000Z,1.000000e+15,True,NaN,NaN
31707,0xfd46e6a48f3b6d7e0fb34f206999e51008d7b1cad632...,0x6be4a7bbb812bfa6a63126ee7b76c8a13529bdb8,0xa6d954d08877f8ce1224f6bfb83484c7d3abf8e9,2018-03-23T09:26:30.000Z,2.000000e+15,True,NaN,NaN
31708,0xfd5264d8b11faa1ca1812a99b250d5dbe66ce37478d8...,0x514910771af9ca656af840dff83e8264ecf986ca,0x49d33037fb37bd8f4a4817bf425540c2d5e4d30e,2021-08-22T23:39:08.000Z,0.000000e+00,NaN,True,NaN
31709,0xfd828fe879d0bf1285986131cf28a9535d7daf9f8980...,0x5fdcca53617f4d2b9134b29090c87d01058e27e9,0xc930830064ca8d752cecb3373197ce8673b21c22,2021-08-23T06:13:19.000Z,3.350000e+17,NaN,True,NaN


In [7]:
G = nx.from_pandas_edgelist(
    tx,
    source='from_address',
    target='to_address',
    edge_key="transaction_hash",
    edge_attr=["value", "block_timestamp", "nft_details"],
    create_using=nx.MultiDiGraph(),
)

NODE_MAPPING = {
    '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b': 'Opensea Exchange',
    '0xa3a4548b39da96eb065ff91811ca30da40431c0d': 'Nate' #main wallet with CryptoPunk 3501
}

def map_node(n):
    if n in NODE_MAPPING:
        return NODE_MAPPING[n]
    else:
        return n

In [8]:
def findPaths(G,u,n):
    if n==0:
        return [[u]]
    paths = [[u]+path for neighbor in G.neighbors(u) for path in findPaths(G,neighbor,n-1)]
    return paths

def find_cycles(G,u,n):
    paths = findPaths(G,u,n)
    return [tuple(path) for path in paths if (path[-1] == u) and sum(x ==u for x in path) == 2]



In [9]:
session_active = False
suspicision_sessions = []
session_id = 0

session_holder = []
nate_outflow = 0
nate_inflow = 0
opensea_buy = False
opensea_sell = False

def reset_session():
    global session_holder
    global nate_outflow
    global nate_inflow
    global opensea_buy
    global opensea_sell
    if (nate_outflow > 0 and nate_inflow > 0 and opensea_buy and opensea_sell):
        suspicision_sessions.append(copy.deepcopy(session_holder))
    session_holder = []
    nate_outflow = 0
    nate_inflow = 0
    opensea_buy = False
    opensea_sell = False

progress_counter = 0

for size in [2, 3]:
    cycles = find_cycles(G,'0xa3a4548b39da96eb065ff91811ca30da40431c0d', size)
    for c in cycles:
        if '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b' not in c:
            progress_counter += 1
            if (progress_counter %5 == 0):
                print(progress_counter)
            reset_session()
            prev_time = pendulum.now()
            sub = list(G.subgraph(list(c) + ['0x7be8076f4ea4a4ad08075c2508e481d6c946d12b']).edges(data=True))
            sub.sort(key=lambda x: x[2]['block_timestamp'])

            for e in sub:
                cur_time = pendulum.parse(e[2]['block_timestamp'][0:-5])
                diff_time = (cur_time - prev_time).in_hours()

                if not session_active and map_node(e[0]) == 'Nate':
                    session_active = True
                    session_holder.append(e)
                    session_id += 1
                    nate_outflow += e[2]['value']/1e18
                elif session_active and diff_time > 36:
                    if map_node(e[0]) == 'Nate':
                        reset_session()
                        session_holder.append(e)
                        session_id += 1
                        nate_outflow += e[2]['value']/1e18
                    else:
                        reset_session()
                        session_active = False
                elif session_active:
                    session_holder.append(e)
                    if map_node(e[0]) == 'Nate':
                        nate_outflow += e[2]['value']/1e18
                    elif map_node(e[1]) == 'Nate':
                        nate_inflow += e[2]['value']/1e18
                    elif map_node(e[0]) == 'Opensea Exchange':
                        opensea_sell = True
                    elif map_node(e[1]) == 'Opensea Exchange':
                        opensea_buy = True

        #         print('{}->{}, {:0.3f} at {}, diff {} hours, session {} {}, num_sus {}'.format(
        #             map_node(e[0]), map_node(e[1]), e[2]['value']/1e18, e[2]['block_timestamp'][0:-5], diff_time, session_id, session_active, len(suspicision_sessions)
        #         ))
                prev_time = cur_time

print('{} suspicious sessions found'.format(len(suspicision_sessions)))

5
10
15
20
25
30
35
40
45
53 suspicious sessions found


In [10]:
tmp = [json.dumps(s) for s in suspicision_sessions]
deduped_sessions = [json.loads(s) for s in list(set(tmp))]
len(deduped_sessions)

18

In [31]:
for e in deduped_sessions[4]:
    if not pd.isnull(e[2]['nft_details']):
        if map_node(e[1]) == 'Opensea Exchange':
            buysell = 'bought'
        else:
            buysell = 'sold'
        tail = '{} {} {}'.format(e[2]['block_timestamp'][0:-5], buysell, e[2]['nft_details'])
    else:
        tail = e[2]['block_timestamp'][0:-5]
    print('{}->{}, {:0.3f}ETH at {}'.format(map_node(e[0]), map_node(e[1]), e[2]['value']/1e18, tail))

Nate->0x3ff308e7b0dfc0724900be69d61c71395dbce97a, 5.000ETH at 2021-07-17T11:06:42
0x3ff308e7b0dfc0724900be69d61c71395dbce97a->Opensea Exchange, 4.700ETH at 2021-07-17T11:07:03 bought BoredApeYachtClub-5462
0x3ff308e7b0dfc0724900be69d61c71395dbce97a->Opensea Exchange, 0.000ETH at 2021-07-17T12:01:56
Opensea Exchange->0x3ff308e7b0dfc0724900be69d61c71395dbce97a, 4.800ETH at 2021-07-17T14:15:11 sold BoredApeYachtClub-5462
0x3ff308e7b0dfc0724900be69d61c71395dbce97a->Nate, 0.000ETH at 2021-07-17T15:06:25
0x3ff308e7b0dfc0724900be69d61c71395dbce97a->Nate, 5.399ETH at 2021-07-17T15:25:03


In [459]:
htmlTemplate = Template(
"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
    <meta charset="utf-8">
    <title>Force Layout with multiple labelled edges between nodes</title>
    <style type="text/css">
    @import url(https://fonts.googleapis.com/css?family=Open+Sans);

    #graph-div {
      background: #384d54;
      color: white;
    }
    text {
        fill: white;
        font-family: 'Open Sans';
    }
    circle {
      stroke: white;
      stroke-width: 1.5px;
    }
    path.link, path.textpath {
        fill: none;
        stroke: #cccccc;
        stroke-width: 0.5px;
    }
    path.invis {
        fill: none;
        stroke-width: 0;
    }
    </style>
    </head>
    <body>
    <div id="graph-div"></div>
    <script src="https://d3js.org/d3.v3.min.js" charset="utf-8"></script>
    <script src="https://code.jquery.com/jquery-2.2.4.min.js"></script>
    <script type="text/javascript">

        var data = $data;

        function myGraph() {
          this.addNode = function (n) {
              if (!findNode(n.id)) {
                  nodes.push({"id": n.id, "label": n.label});
                  update();
              }
          };

          this.addLink = function (source, target, value) {
              links.push({"source": findNode(source.id), "target": findNode(target.id), "value": value});
              update();
          };

          this.initialize = function() {
            data.edges.forEach(function(d) {
                graph.addNode(d.source);
                graph.addNode(d.target);
                graph.addLink(d.source, d.target, d.value);
            });
          };

          var findNode = function (nodeId) {
              for (var i in nodes) {
                  if (nodes[i].id === nodeId) {
                    return nodes[i];
                  }
              };
          };

          var countSiblingLinks = function(source, target) {
              var count = 0;
              for(var i = 0; i < links.length; ++i){
                  if( (links[i].source.id == source.id && links[i].target.id == target.id) || (links[i].source.id == target.id && links[i].target.id == source.id) )
                      count++;
              };
              return count;
          };

          var getSiblingLinks = function(source, target) {
              var siblings = [];
              for(var i = 0; i < links.length; ++i){
                  if( (links[i].source.id == source.id && links[i].target.id == target.id) || (links[i].source.id == target.id && links[i].target.id == source.id) )
                      siblings.push(links[i].value);
              };
              return siblings;
          };

          var linkDistance = 400;

          var margin = {top: 20, right: 20, bottom: 20, left: 20},
          w = 1200 - margin.left - margin.right,
          h = 600 - margin.top - margin.bottom;
          middle = w/2;

          var colors = d3.scale.category20();

          var svg = d3.select("#graph-div")
                .append("svg:svg")
                  .attr("width", w)
                  .attr("height", h)
                  .style("z-index", -10)
                  .attr("id", "svg");

              svg.append('svg:defs').selectAll('marker')
                  .data(['end'])
                  .enter()
                .append('svg:marker')
                  .attr({'id': "arrowhead",
                         'viewBox':'0 -5 10 10',
                         'refX': 22,
                         'refY': 0,
                         'orient':'auto',
                         'markerWidth': 20,
                         'markerHeight': 20,
                         'markerUnits': "strokeWidth",
                         'xoverflow':'visible'})
                .append('svg:path')
                  .attr('d', 'M0,-5L10,0L0,5')
                  .attr('fill', '#ccc');

            var force = d3.layout.force();

            var nodes = force.nodes(),
                links = force.links();

            var update = function () {

                var path = svg.selectAll("path.link")
                        .data(force.links());

                    path.enter().append("svg:path")
                        .attr("id", function (d) {
                            return d.source.id + "-" + d.value + "-" + d.target.id;
                        })
                        .attr("class", "link")
                        .attr('marker-end','url(#arrowhead)');

                    path.exit().remove();

                var pathInvis = svg.selectAll("path.invis")
                        .data(force.links());

                    pathInvis.enter().append("svg:path")
                        .attr("id", function (d) {
                            return "invis_" + d.source.id + "-" + d.value + "-" + d.target.id;
                        })
                        .attr("class", "invis");

                    pathInvis.exit().remove();

                var pathLabel = svg.selectAll(".pathLabel")
                        .data(force.links());

                    pathLabel.enter().append("g").append("svg:text")
                        .attr("class", "pathLabel")
                        .append("svg:textPath")
                          .attr("startOffset", "50%")
                          .attr("text-anchor", "middle")
                          .attr("xlink:href", function(d) { return "#invis_" + d.source.id + "-" + d.value + "-" + d.target.id; })
                          .style("fill", "#cccccc")
                          .style("font-size", 10)
                          .text(function(d) { return d.value; });

                var node = svg.selectAll("g.node")
                        .data(force.nodes());

                var nodeEnter = node.enter().append("g")
                        .attr("class", "node")
                        .call(force.drag);

                    nodeEnter.append("svg:circle")
                            .attr("r", 10)
                            .attr("id", function (d) {
                                return "Node;" + d.id;
                            })
                            .attr("class", "nodeStrokeClass")
                            .attr("fill", "#0db7ed")

                    nodeEnter.append("svg:text")
                            .attr("class", "textClass")
                            .attr("x", 20)
                            .attr("y", ".31em")
                            .text(function (d) {
                                return d.label;
                            });

                    node.exit().remove();

                    function arcPath(leftHand, d) {
                        var x1 = leftHand ? d.source.x : d.target.x,
                            y1 = leftHand ? d.source.y : d.target.y,
                            x2 = leftHand ? d.target.x : d.source.x,
                            y2 = leftHand ? d.target.y : d.source.y,
                            dx = x2 - x1,
                            dy = y2 - y1,
                            dr = Math.sqrt(dx * dx + dy * dy),
                            drx = dr,
                            dry = dr,
                            sweep = leftHand ? 0 : 1;
                            siblingCount = countSiblingLinks(d.source, d.target)
                            xRotation = 0,
                            largeArc = 0;

                            if (siblingCount > 1) {
                                var siblings = getSiblingLinks(d.source, d.target);
                                var arcScale = d3.scale.ordinal()
                                                        .domain(siblings)
                                                        .rangePoints([1, siblingCount]);
                                drx = drx/(1 + (1/siblingCount) * (arcScale(d.value) - 1));
                                dry = dry/(1 + (1/siblingCount) * (arcScale(d.value) - 1));
                            }

                        return "M" + x1 + "," + y1 + "A" + drx + ", " + dry + " " + xRotation + ", " + largeArc + ", " + sweep + " " + x2 + "," + y2;
                    }

                    force.on("tick", function(e) {
                        var q = d3.geom.quadtree(nodes),
                            i = 0,
                            n = nodes.length,
                            k = .1 * e.alpha;

                        while (++i < n) q.visit(collide(nodes[i]));

                        node.attr("transform", function (d) {
                            return "translate(" + d.x + "," + d.y + ")";
                        });

                        path.attr("d", function(d) {
                            return arcPath(true, d);
                        });

                        pathInvis.attr("d", function(d) {
                            return arcPath(d.source.x < d.target.x, d);
                        });
                    });

                    force
                            .charge(-1000)
                            .friction(0.5)
                            .linkDistance(linkDistance)
                            .size([w, h])
                            .start();
                    keepNodesOnTop();

            }

            update();

            function collide(node) {
              var r = node.radius + 16,
                  nx1 = node.x - r,
                  nx2 = node.x + r,
                  ny1 = node.y - r,
                  ny2 = node.y + r;
              return function(quad, x1, y1, x2, y2) {
                if (quad.point && (quad.point !== node)) {
                  var x = node.x - quad.point.x,
                      y = node.y - quad.point.y,
                      l = Math.sqrt(x * x + y * y),
                      r = node.radius + quad.point.radius;
                  if (l < r) {
                    l = (l - r) / l * .5;
                    node.x -= x *= l;
                    node.y -= y *= l;
                    quad.point.x += x;
                    quad.point.y += y;
                  }
                }
                return x1 > nx2 || x2 < nx1 || y1 > ny2 || y2 < ny1;
              };
            }
        }

    function drawGraph() {
        graph = new myGraph();
        graph.initialize();
    }

    drawGraph();

    function keepNodesOnTop() {
        $$(".nodeStrokeClass").each(function( index ) {
            var gNode = this.parentNode;
            gNode.parentNode.appendChild(gNode);
        });
    }

    </script>
    </body>
    </html>
""")

In [490]:
json_data = {
    "edges":
    [
        {
            "source": {"id": e[0], "label": map_node(e[0])},
            "target": {'id': e[1], "label": map_node(e[1])},
            "value": '{:0.3f} at {}'.format(e[2]['value']/1e18, e[2]['block_timestamp'][0:-5])
              
        }
        for e in deduped_sessions[12]
    ]
}

display(HTML(htmlTemplate.substitute(data=json.dumps(json_data))))

In [30]:
min([e[2]['block_timestamp'] for t in deduped_sessions for e in t])

'2021-05-07T21:47:44.000Z'